In [1]:
import           Control.Monad.Trans.State.Strict
import qualified Data.IntMap.Strict  as M
import qualified Data.Vector         as V

import           Control.Monad (forM_, when)
import           Data.Functor (void)
import           Data.Maybe (isNothing, fromJust)
import           Prelude hiding (lookup)

In [2]:
graph = M.fromList
    [ (1, [2])
    , (2, [3])
    , (3, [1])
    , (4, [2, 3, 5])
    , (5, [4, 6])
    , (6, [7, 3])
    , (7, [6])
    , (8, [5, 7, 8])
    ]

In [6]:
data TState = TState
    { index    :: Int
    , stack    :: [Int]
    , stackSet :: V.Vector Bool
    , stackTop :: Maybe Int
    , indices  :: V.Vector (Maybe Int)
    , lowlinks :: V.Vector (Maybe Int)
    , output   :: [[Int]]
    } deriving (Eq, Show)
  
data Event
    = IncrementIndex
    | InsertIndex   Int Int
    | InsertLowlink Int Int
    | Push Int
    | Pop
    | OutputSCC [Int]
    deriving (Eq, Show)

data EState = EState
    { current :: TState
    , history :: [Event]
    } deriving (Eq, Show)

insert :: V.Vector a -> Int -> a -> V.Vector a
insert vector index value = vector V.// [(index, value)]

apply :: Event -> TState -> TState
apply event state = case event of
    IncrementIndex    -> state { index    = index state + 1}
    InsertIndex k v   -> state { indices  = insert (indices  state) k (Just v) }
    InsertLowlink k v -> state { lowlinks = insert (lowlinks state) k (Just v) }
    Push i            -> state
        { stack    = i : (stack state) 
        , stackSet = insert (stackSet state) i True
        }
    Pop               -> let i = head (stack state) in state
        { stack    = tail (stack state)
        , stackSet = insert (stackSet state) i False
        , stackTop = Just i
        }
    OutputSCC scc     -> state { output = output state ++ [scc] }

initial = TState 0 [] newVector' Nothing newVector newVector []
     where 
        size = (fst $ M.findMax graph) + 1
        newVector  = V.replicate size Nothing
        newVector' = V.replicate size False

emit :: Event -> State EState ()
emit event = void . modify' $ \s -> s
    { current = apply event (current s) 
    , history = event : (history s)
    }

incrementIndex    = emit   IncrementIndex
insertIndex   k v = emit $ InsertIndex k v
insertLowlink k v = emit $ InsertLowlink k v
push i            = emit $ Push i
pop               = emit   Pop
outputSCC scc     = emit $ OutputSCC scc

In [7]:
tarjan :: M.IntMap [Int] -> ([[Int]], EState)
tarjan graph = flip runState (EState initial []) $ do
    forM_ (M.keys graph) $ \v -> do
        is <- indices <$> gets current
        when (isNothing (is V.! v)) $ strongConnect v graph
    output <$> gets current

strongConnect :: Int -> M.IntMap [Int] -> State EState ()
strongConnect v graph = do
    incrementIndex
    i <- index <$> gets current
    insertIndex   v i
    insertLowlink v i
    push v
    forM_ (graph M.! v) $ \w -> do
        is <- indices <$> gets current
        case is V.! w of
            Nothing -> do
                strongConnect w graph
                ls <- lowlinks <$> gets current
                let vLowlink = fromJust (ls V.! v)
                let wLowlink = fromJust (ls V.! w)
                insertLowlink v (min vLowlink wLowlink)
            Just wIndex -> do
                sSet <- stackSet <$> gets current
                when (sSet V.! w) $ do
                    ls <- lowlinks <$> gets current
                    let vLowlink = fromJust (ls V.! v)
                    insertLowlink v (min vLowlink wIndex)
    s <- gets current
    let vLowlink = fromJust (lowlinks s V.! v)
    let vIndex   = fromJust (indices  s V.! v)
    when (vLowlink == vIndex) $ do
        scc <- addSCC v []
        outputSCC scc

addSCC :: Int -> [Int] -> State EState [Int]
addSCC v scc = do
    pop
    w <- fromJust . stackTop <$> gets current
    let scc' = w:scc
    if w == v
        then return scc'
        else addSCC v scc'

In [10]:
complete = tarjan graph
void $ traverse print (reverse . history . snd $ complete)

IncrementIndex
InsertIndex 1 1
InsertLowlink 1 1
Push 1
IncrementIndex
InsertIndex 2 2
InsertLowlink 2 2
Push 2
IncrementIndex
InsertIndex 3 3
InsertLowlink 3 3
Push 3
InsertLowlink 3 1
InsertLowlink 2 1
InsertLowlink 1 1
Pop
Pop
Pop
OutputSCC [1,2,3]
IncrementIndex
InsertIndex 4 4
InsertLowlink 4 4
Push 4
IncrementIndex
InsertIndex 5 5
InsertLowlink 5 5
Push 5
InsertLowlink 5 4
IncrementIndex
InsertIndex 6 6
InsertLowlink 6 6
Push 6
IncrementIndex
InsertIndex 7 7
InsertLowlink 7 7
Push 7
InsertLowlink 7 6
InsertLowlink 6 6
Pop
Pop
OutputSCC [6,7]
InsertLowlink 5 4
InsertLowlink 4 4
Pop
Pop
OutputSCC [4,5]
IncrementIndex
InsertIndex 8 8
InsertLowlink 8 8
Push 8
InsertLowlink 8 8
Pop
OutputSCC [8]